In [1]:
!rm -rf /kaggle/working/rwkv_online/
!git clone https://github.com/v3ucn/rwkv_online.git
%cd rwkv_online
!pip install gradio
!pip install huggingface_hub
!pip install pynvml
!pip install rwkv
!pip install Ninja

Cloning into 'rwkv_online'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 24 (delta 6), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (24/24), 17.80 KiB | 2.54 MiB/s, done.
/kaggle/working/rwkv_online
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 59.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 70.6 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 8.1 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=5584 sha256=624388418095b6f16f4ebf914083a9c7904838098ac6c024aa7026ab5cb1f276
  Stored in directory: /root/.cache/pip/wheels/bd/65/9a/671fc6dcde07d4418df0c592f8df512b26d7a0029c2a23dd81
Successfully built ffmpy
  Attempting uninstall: 

In [ ]:
import torch

print(torch.cuda.is_available())

print(torch.cuda.device_count())

In [2]:
import gradio as gr
import os, gc, copy, torch
from datetime import datetime
from huggingface_hub import hf_hub_download
from pynvml import *

# Flag to check if GPU is present
HAS_GPU = True

# Model title and context size limit
ctx_limit = 2000
title = "RWKV-5-H-World-3B"
model_file = "rwkv-5-h-world-3B"

# Get the GPU count
try:
    nvmlInit()
    GPU_COUNT = nvmlDeviceGetCount()
    if GPU_COUNT > 0:
        HAS_GPU = True
        gpu_h = nvmlDeviceGetHandleByIndex(0)
except NVMLError as error:
    print(error)


os.environ["RWKV_CUDA_ON"] = '1'
MODEL_STRAT = "cuda bf16"

# Load the model accordingly
from rwkv.model import RWKV
model_path = hf_hub_download(repo_id="a686d380/rwkv-5-h-world", filename=f"{model_file}.pth")
model = RWKV(model=model_path, strategy=MODEL_STRAT)
from rwkv.utils import PIPELINE, PIPELINE_ARGS
pipeline = PIPELINE(model, "rwkv_vocab_v20230424")

Using /root/.cache/torch_extensions/py310_cu121 as PyTorch extensions root...
Creating extension directory /root/.cache/torch_extensions/py310_cu121/wkv_cuda...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py310_cu121/wkv_cuda/build.ninja...
Building extension module wkv_cuda...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/4] c++ -MMD -MF gemm_fp16_cublas.o.d -DTORCH_EXTENSION_NAME=wkv_cuda -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1013\" -isystem /opt/conda/lib/python3.10/site-packages/torch/include -isystem /opt/conda/lib/python3.10/site-packages/torch/include/torch/csrc/api/include -isystem /opt/conda/lib/python3.10/site-packages/torch/include/TH -isystem /opt/conda/lib/python3.10/site-packages/torch/include/THC -isystem /usr/local/cuda/include -isystem /opt/conda/include/python3.10 -D_GLIBCXX_USE_CXX11_ABI=1 -fPIC -std=c++17 -c /opt/conda/lib/python3.10/site-packages/rwkv/cuda/gemm_fp16_cublas.cpp -o gemm_fp16_cublas.o 
[2/4] c++ -MMD -MF wrapper.o.d -DTORCH_EXTENSION_NAME=wkv_cuda -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1013\" -isystem /opt/conda/lib/python3.10/site-packages/torch/include -isystem /opt/conda/lib/python3.10/

Loading extension module wkv_cuda...


rwkv-5-h-world-3B.pth:   0%|          | 0.00/6.13G [00:00<?, ?B/s]

RWKV_JIT_ON 1 RWKV_CUDA_ON 1 RESCALE_LAYER 0

Loading /root/.cache/huggingface/hub/models--a686d380--rwkv-5-h-world/snapshots/d77242c17d8715ebdabc1e4e252038f8a39c45cc/rwkv-5-h-world-3B.pth ...
Model detected: v5.2
Strategy: (total 32+1=33 layers)
* cuda [bfloat16, bfloat16], store 33 layers
0-cuda-bfloat16-bfloat16 1-cuda-bfloat16-bfloat16 2-cuda-bfloat16-bfloat16 3-cuda-bfloat16-bfloat16 4-cuda-bfloat16-bfloat16 5-cuda-bfloat16-bfloat16 6-cuda-bfloat16-bfloat16 7-cuda-bfloat16-bfloat16 8-cuda-bfloat16-bfloat16 9-cuda-bfloat16-bfloat16 10-cuda-bfloat16-bfloat16 11-cuda-bfloat16-bfloat16 12-cuda-bfloat16-bfloat16 13-cuda-bfloat16-bfloat16 14-cuda-bfloat16-bfloat16 15-cuda-bfloat16-bfloat16 16-cuda-bfloat16-bfloat16 17-cuda-bfloat16-bfloat16 18-cuda-bfloat16-bfloat16 19-cuda-bfloat16-bfloat16 20-cuda-bfloat16-bfloat16 21-cuda-bfloat16-bfloat16 22-cuda-bfloat16-bfloat16 23-cuda-bfloat16-bfloat16 24-cuda-bfloat16-bfloat16 25-cuda-bfloat16-bfloat16 26-cuda-bfloat16-bfloat16 27-cuda-bfloat16

Using /root/.cache/torch_extensions/py310_cu121 as PyTorch extensions root...
Creating extension directory /root/.cache/torch_extensions/py310_cu121/rwkv5...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py310_cu121/rwkv5/build.ninja...
Building extension module rwkv5...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/3] c++ -MMD -MF rwkv5_op.o.d -DTORCH_EXTENSION_NAME=rwkv5 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1013\" -isystem /opt/conda/lib/python3.10/site-packages/torch/include -isystem /opt/conda/lib/python3.10/site-packages/torch/include/torch/csrc/api/include -isystem /opt/conda/lib/python3.10/site-packages/torch/include/TH -isystem /opt/conda/lib/python3.10/site-packages/torch/include/THC -isystem /usr/local/cuda/include -isystem /opt/conda/include/python3.10 -D_GLIBCXX_USE_CXX11_ABI=1 -fPIC -std=c++17 -c /opt/conda/lib/python3.10/site-packages/rwkv/cuda/rwkv5_op.cpp -o rwkv5_op.o 
[2/3] /usr/local/cuda/bin/nvcc  -DTORCH_EXTENSION_NAME=rwkv5 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1013\" -isystem /opt/conda/lib/python3.10/site-packages/torch/include -isystem /opt/conda/lib/python3.10/site-packages/torch/include/t

Loading extension module rwkv5...


In [ ]:
# 开启推理页面
%cd /kaggle/working/rwkv_online/
!npm install -g localtunnel
import subprocess
import threading
import time
import socket
import urllib.request
def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock= socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()

        from colorama import Fore, Style
    print (Fore.GREEN + "\nIP: ", Fore. RED, urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"), "\n", Style. RESET_ALL)
    p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
    for line in p.stdout:
        print(line.decode(), end='')
threading.Thread (target=iframe_thread, daemon=True, args=(9872,)).start()

!python  ./webui.py

/kaggle/working/rwkv_online
m#################.] / reify:debug: http fetch GET 200 https://registry.npmjs.o
added 22 packages in 2s

3 packages are looking for funding
  run `npm fund` for details
npm notice 
npm notice New minor version of npm available! 10.1.0 -> 10.5.0
npm notice Changelog: https://github.com/npm/cli/releases/tag/v10.5.0
npm notice Run npm install -g npm@10.5.0 to update!
npm notice 
Running on local URL:  http://0.0.0.0:9872

IP:   35.238.86.110 
 
your url is: https://angry-mugs-refuse.loca.lt
Running on public URL: https://2a7d4144ed321ff3b3.gradio.live
